# Import packages

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)
from scipy.stats import levy_stable

from datetime import datetime
from scipy.stats import kstest
from scipy.stats import jarque_bera
# from arch.unitroot import ADF
from scipy.stats import kurtosis
from scipy.stats import skew
# from arch import arch_model

import pickle

import ta

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import webbrowser
from datetime import datetime, timedelta

import random

plt.style.use('classic')
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 300)

# %config inlinebackend.figure_format = 'svg'

In [ ]:
import sys

sys.path.insert(0, 'D:/Intraday_trading')

In [ ]:
from src.support import *
from src.backtest import *
from src.models import *
from src.models_support import *

# Import and pre-processing data

## Interval data

In [ ]:
%%time

interval_data = pd.read_pickle('D:/Intraday_trading/Training/Data/XAUUSD_M15_1.pkl')

interval_data = interval_data.set_index('DATE_TIME')
interval_data.index = pd.to_datetime(interval_data.index)

interval_data['DATE'] = pd.to_datetime(interval_data['DATE'])
interval_data['OPEN'] = interval_data['OPEN']
interval_data['HIGH'] = interval_data['HIGH']
interval_data['LOW'] = interval_data['LOW']
interval_data['CLOSE'] = interval_data['CLOSE']


### Prepare_df

In [ ]:
df_4_hour = prepare_df(df = interval_data, timeframe = '4H', add_indicators = True)
df_1_day = prepare_df(df = interval_data, timeframe = '1D', add_indicators = True)
df_15_min = prepare_df(df = interval_data, timeframe = '15min', add_indicators = True)

df_1_day['WHOLE_RANGE'] = df_1_day['HIGH'] - df_1_day['LOW']
df_4_hour['WHOLE_RANGE'] = df_4_hour['HIGH'] - df_4_hour['LOW']
df_15_min['WHOLE_RANGE'] = df_15_min['HIGH'] - df_15_min['LOW']

df_1_day['GRP_WHOLE_RANGE'] = pd.qcut(df_1_day['WHOLE_RANGE'], 10)
df_4_hour['GRP_WHOLE_RANGE'] = pd.qcut(df_4_hour['WHOLE_RANGE'], 10)
df_15_min['GRP_WHOLE_RANGE'] = pd.qcut(df_15_min['WHOLE_RANGE'], 10)

df_1_day['GRP_BODY'] = pd.qcut(df_1_day['BODY'], 10)
df_4_hour['GRP_BODY'] = pd.qcut(df_4_hour['BODY'], 10)
df_15_min['GRP_BODY'] = pd.qcut(df_15_min['BODY'], 10)

df_1_day['YEAR'] = df_1_day.index.strftime('%Y')
df_1_day['WEEK'] = df_1_day.index.strftime('%Y%W')
df_1_day['MONTH'] = df_1_day.index.strftime('%Y%m')

df_4_hour['YEAR'] = df_4_hour.index.strftime('%Y')
df_4_hour['WEEK'] = df_4_hour.index.strftime('%Y%W')
df_4_hour['MONTH'] = df_4_hour.index.strftime('%Y%m')

df_15_min['YEAR'] = df_15_min.index.strftime('%Y')
df_15_min['WEEK'] = df_15_min.index.strftime('%Y%W')
df_15_min['MONTH'] = df_15_min.index.strftime('%Y%m')



In [ ]:
df_15_min.shape, df_4_hour.shape, df_1_day.shape, 

In [ ]:
df_1_day.columns

In [ ]:
# df_15_min.index[0], df_15_min.index[-1]

In [ ]:
# plot_df(df_1_day, 
#         path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
#         open_tab = False)

# Strategy

In [ ]:
interval_is = interval_data[interval_data.index < datetime(2023, 1, 1)]
interval_os = interval_data[interval_data.index >= datetime(2023, 1, 1)]


In [ ]:
df_4_hour_is = prepare_df(df = interval_is, timeframe = '4H', add_indicators = False)
df_1_day_is = prepare_df(df = interval_is, timeframe = '1D', add_indicators = False)
df_15_min_is = prepare_df(df = interval_is, timeframe = '15min', add_indicators = False)

df_1_day_is['WHOLE_RANGE'] = df_1_day_is['HIGH'] - df_1_day_is['LOW']
df_4_hour_is['WHOLE_RANGE'] = df_4_hour_is['HIGH'] - df_4_hour_is['LOW']
df_15_min_is['WHOLE_RANGE'] = df_15_min_is['HIGH'] - df_15_min_is['LOW']

df_1_day_is['GRP_WHOLE_RANGE'] = pd.qcut(df_1_day_is['WHOLE_RANGE'], 10)
df_4_hour_is['GRP_WHOLE_RANGE'] = pd.qcut(df_4_hour_is['WHOLE_RANGE'], 10)
df_15_min_is['GRP_WHOLE_RANGE'] = pd.qcut(df_15_min_is['WHOLE_RANGE'], 10)

df_1_day_is['GRP_BODY'] = pd.qcut(df_1_day_is['BODY'], 10)
df_4_hour_is['GRP_BODY'] = pd.qcut(df_4_hour_is['BODY'], 10)
df_15_min_is['GRP_BODY'] = pd.qcut(df_15_min_is['BODY'], 10)

df_1_day_is['YEAR'] = df_1_day_is.index.strftime('%Y')
df_1_day_is['WEEK'] = df_1_day_is.index.strftime('%Y%W')
df_1_day_is['MONTH'] = df_1_day_is.index.strftime('%Y%m')
df_1_day_is['DATE'] = df_1_day_is.index.strftime('%Y%m%d')

df_4_hour_is['YEAR'] = df_4_hour_is.index.strftime('%Y')
df_4_hour_is['WEEK'] = df_4_hour_is.index.strftime('%Y%W')
df_4_hour_is['MONTH'] = df_4_hour_is.index.strftime('%Y%m')
df_4_hour_is['DATE'] = df_4_hour_is.index.strftime('%Y%m%d')

df_15_min_is['YEAR'] = df_15_min_is.index.strftime('%Y')
df_15_min_is['WEEK'] = df_15_min_is.index.strftime('%Y%W')
df_15_min_is['MONTH'] = df_15_min_is.index.strftime('%Y%m')
df_15_min_is['DATE'] = df_15_min_is.index.strftime('%Y%m%d')

#-----------------------------------------------------

df_4_hour_os = prepare_df(df = interval_os, timeframe = '4H', add_indicators = False)
df_1_day_os = prepare_df(df = interval_os, timeframe = '1D', add_indicators = False)
df_15_min_os = prepare_df(df = interval_os, timeframe = '15min', add_indicators = False)

df_1_day_os['WHOLE_RANGE'] = df_1_day_os['HIGH'] - df_1_day_os['LOW']
df_4_hour_os['WHOLE_RANGE'] = df_4_hour_os['HIGH'] - df_4_hour_os['LOW']
df_15_min_os['WHOLE_RANGE'] = df_15_min_os['HIGH'] - df_15_min_os['LOW']

df_1_day_os['GRP_WHOLE_RANGE'] = pd.qcut(df_1_day_os['WHOLE_RANGE'], 10)
df_4_hour_os['GRP_WHOLE_RANGE'] = pd.qcut(df_4_hour_os['WHOLE_RANGE'], 10)
df_15_min_os['GRP_WHOLE_RANGE'] = pd.qcut(df_15_min_os['WHOLE_RANGE'], 10)

df_1_day_os['GRP_BODY'] = pd.qcut(df_1_day_os['BODY'], 10)
df_4_hour_os['GRP_BODY'] = pd.qcut(df_4_hour_os['BODY'], 10)
df_15_min_os['GRP_BODY'] = pd.qcut(df_15_min_os['BODY'], 10)

df_1_day_os['YEAR'] = df_1_day_os.index.strftime('%Y')
df_1_day_os['WEEK'] = df_1_day_os.index.strftime('%Y%W')
df_1_day_os['MONTH'] = df_1_day_os.index.strftime('%Y%m')

df_4_hour_os['YEAR'] = df_4_hour_os.index.strftime('%Y')
df_4_hour_os['WEEK'] = df_4_hour_os.index.strftime('%Y%W')
df_4_hour_os['MONTH'] = df_4_hour_os.index.strftime('%Y%m')

df_15_min_os['YEAR'] = df_15_min_os.index.strftime('%Y')
df_15_min_os['WEEK'] = df_15_min_os.index.strftime('%Y%W')
df_15_min_os['MONTH'] = df_15_min_os.index.strftime('%Y%m')


In [ ]:
df_1_day_is.shape, df_1_day_os.shape

In [ ]:
class TA_strat_v1(Base_Alpha):
    def __init__(self, config):
        super().__init__(
            strat = self._TA_strat_v1,
            prepare_data = self._prepare_data
        )  # Pass the strategy function to the parent constructor
        
        '''
        config = {
            'RSI_PARAMS': {'INPUT_PARAM': 14, 'CUTOFF_BUY': 30, 'CUTOFF_SELL': 70},
            'BB_PARAMS': {'INPUT_MEAN': 50, 'INPUT_SD': 2, 'BB_CUTOFF': 5},
            'EMA_1_PARAMS': {'INPUT_PARAM': 50},
            'EMA_2_PARAMS': {'INPUT_PARAM': 200},
            'WEIGHT_DAILY': 5,
            'WEIGHT_4HOUR': 3,
            'WEIGHT_15MIN': 3,
            'LOOKBACK': 40,
            'base_SL': 10, 
            'base_TP': 20,
            'DECISION_CUTOFF': {'ENTRY_BUY_CUTOFF': 0.5, 'ENTRY_SELL_CUTOFF': 0.5}
        }
        '''
        self.RSI_PARAMS = config['RSI_PARAMS']
        self.BB_PARAMS = config['BB_PARAMS']
        self.EMA_1_PARAMS = config['EMA_1_PARAMS']
        self.EMA_2_PARAMS = config['EMA_2_PARAMS']
        self.WEIGHT_DAILY = config['WEIGHT_DAILY']
        self.WEIGHT_4HOUR = config['WEIGHT_4HOUR']
        self.WEIGHT_15MIN = config['WEIGHT_15MIN']
        self.LOOKBACK = config['LOOKBACK']
        self.base_SL = config['base_SL']
        self.base_TP = config['base_TP']
        self.DECISION_CUTOFF = config['DECISION_CUTOFF']
        
        
    def _prepare_data(self, df_observe):

        # RSI
        df_observe['RSI'] = ta.momentum.RSIIndicator(df_observe['CLOSE'], window = self.RSI_PARAMS['INPUT_PARAM']).rsi()
        
        # Bollinger band
        df_observe[f'BB_UPPER_BAND({self.BB_PARAMS["INPUT_MEAN"]})'] = ta.volatility.BollingerBands(df_observe['CLOSE'], 
                                                                                                    window = self.BB_PARAMS['INPUT_MEAN'], 
                                                                                                    window_dev = self.BB_PARAMS['INPUT_SD']).bollinger_hband()
        df_observe[f'BB_LOWER_BAND({self.BB_PARAMS["INPUT_MEAN"]})'] = ta.volatility.BollingerBands(df_observe['CLOSE'], 
                                                                                                    window = self.BB_PARAMS['INPUT_MEAN'], 
                                                                                                    window_dev = self.BB_PARAMS['INPUT_SD']).bollinger_lband()
        
        # Exponential moving average
        df_observe[f'EMA({self.EMA_1_PARAMS["INPUT_PARAM"]})'] = ta.trend.EMAIndicator(df_observe['CLOSE'],
                                                window = self.EMA_1_PARAMS["INPUT_PARAM"]).ema_indicator()
        df_observe[f'EMA({self.EMA_2_PARAMS["INPUT_PARAM"]})'] = ta.trend.EMAIndicator(df_observe['CLOSE'],
                                                window = self.EMA_2_PARAMS["INPUT_PARAM"]).ema_indicator()

        return(df_observe)
    
    def _identify_general_trend(self, df_observe):
        '''
        Identify trend and reversal based ONLY on support, resistance, previous highs and lows
        '''

        # Prepare local max, min

        daily_local_max_indices, daily_local_min_indices = search_extremum(df_observe = df_observe, order = 3)

        # Prepare support, resistance

        sr_arr = [5]
        pr_arr = [4]
        pt_arr = [3]
        max_num_arr = [5]
        cutoff_arr = [1]

        # best_params = search_sr(df_observe, 
        #                         sr_arr = sr_arr,
        #                         pr_arr = pr_arr,
        #                         pt_arr = pt_arr,
        #                         max_num_arr = max_num_arr,
        #                         cutoff_arr = cutoff_arr
        #                 )

        best_params = {
             'sr_range': 5,
             'patience_range': 4,
             'patience_time': 3,
             'max_num_range': 5,
             'cutoff': 1,
        }


        # price_hist = prepare_df_sr(df_observe = df_observe,
        #         sr_range = best_params['sr_range'],
        #         patience_range = best_params['patience_range'],
        #         patience_time = best_params['patience_time'])


        # sup_resist = price_hist[(price_hist['SR_RANK'] <= best_params['max_num_range']) & (price_hist['SR_SCORE'] >= best_params['cutoff'])].index
        dist_to_sr = []
        # for _, sr in enumerate(sup_resist):
        #         dist_to_sr.append(df_observe['CLOSE'][-1] - (sr.left + sr.right)/2)

        period_change = df_observe['CLOSE'][-1] - df_observe['CLOSE'][0]

        num_local_max = len(daily_local_max_indices)
        num_local_min = len(daily_local_min_indices)

        if num_local_max > 0:
                latest_local_max_change = df_observe['CLOSE'][-1] - df_observe.iloc[daily_local_max_indices[-1] + 2, :]['HIGH']
                local_max_change = df_observe.iloc[daily_local_max_indices[-1] + 2, :]['HIGH'] - df_observe.iloc[daily_local_max_indices[0] + 2, :]['HIGH']
        else:
                latest_local_max_change = 0
                local_max_change = 0

        if num_local_min > 0:
                latest_local_min_change = df_observe['CLOSE'][-1] - df_observe.iloc[daily_local_min_indices[-1] + 2, :]['LOW']
                local_min_change = df_observe.iloc[daily_local_min_indices[-1] + 2, :]['LOW'] - df_observe.iloc[daily_local_min_indices[0] + 2, :]['LOW']
        else:
                latest_local_min_change = 0
                local_min_change = 0

        trend_score = (np.abs(period_change) > 10)*np.sign(period_change) + \
                        (np.abs(period_change) > 30)*2*np.sign(period_change) + \
                        (num_local_max <= 3)*np.sign(period_change) + \
                        (num_local_min <= 3)*np.sign(period_change) + \
                        (num_local_max < 2)*2*np.sign(period_change) + \
                        (num_local_min < 2)*2*np.sign(period_change) + \
                        (np.abs(local_max_change) > 5)*np.sign(local_max_change) + \
                        (np.abs(local_max_change) > 10)*2*np.sign(local_max_change) + \
                        (np.abs(local_min_change) > 5)*np.sign(local_min_change) + \
                        (np.abs(local_min_change) > 10)*2*np.sign(local_min_change)
        trend_score /= 15

        pivot_score = (period_change > 0)*(latest_local_max_change < -10) + \
                        (period_change > 0)*(latest_local_max_change < -15)*2 + \
                        (period_change > 0)*(latest_local_min_change < 0) + \
                        (period_change > 0)*(latest_local_min_change < -10)*2 + \
                        (period_change > 0)*(latest_local_min_change < -15)*3 + \
                        (period_change < 0)*(latest_local_min_change > 10) + \
                        (period_change < 0)*(latest_local_min_change > 15)*2 + \
                        (period_change < 0)*(latest_local_max_change < 0) + \
                        (period_change < 0)*(latest_local_max_change < -10)*2 + \
                        (period_change < 0)*(latest_local_max_change < -15)*3
        
        if len(dist_to_sr) >= 2:
            pivot_score += (period_change > 0)*(dist_to_sr[-1] < -10) + \
                        (period_change > 0)*(dist_to_sr[-1] < -15)*2 + \
                        (period_change < 0)*(dist_to_sr[0] > 10) + \
                        (period_change < 0)*(dist_to_sr[0] > 15)*2
            pivot_score /= 12
        else: 
            pivot_score /= 9

        return(trend_score, pivot_score)

    def _TA_strat_v1(self, row):
        try:
            df_1_day_is_observed = df_1_day_is[df_1_day_is.index <= row.name].tail(self.LOOKBACK).copy()
            df_4_hour_is_observed = df_4_hour_is[df_4_hour_is.index <= row.name].tail(self.LOOKBACK).copy()
            df_15_min_is_observed = df_15_min_is[df_15_min_is.index <= row.name].tail(self.LOOKBACK).copy()

            trend_score_daily, pivot_score_daily = self._identify_general_trend(df_1_day_is_observed.copy())
            trend_score_4hour, pivot_score_4hour = self._identify_general_trend(df_4_hour_is_observed.copy())
            trend_score_15min, pivot_score_15min = self._identify_general_trend(df_15_min_is_observed.copy())

            entry_buy_score = (
                    (trend_score_daily > 0.5) * self.WEIGHT_DAILY + \
                    ((trend_score_daily < -0.5) & (pivot_score_daily > 0.5)) * self.WEIGHT_DAILY + \
                    (trend_score_4hour > 0.3) * self.WEIGHT_4HOUR + \
                    ((trend_score_4hour < -0.3) & (pivot_score_4hour > 0.3)) * self.WEIGHT_4HOUR + \
                    (trend_score_15min > 0) * self.WEIGHT_15MIN + \
                    1
                ) * \
                (
                    (row['RSI'] < self.RSI_PARAMS['CUTOFF_BUY']) + \
                    (
                        (row['CLOSE'] <= row[f'EMA({self.EMA_1_PARAMS["INPUT_PARAM"]})']) & \
                        (row['CLOSE'] >= row[f'EMA({self.EMA_2_PARAMS["INPUT_PARAM"]})'])
                    ) + \
                    (np.abs(row['CLOSE'] - row[f'BB_LOWER_BAND({self.BB_PARAMS["INPUT_MEAN"]})']) < self.BB_PARAMS['BB_CUTOFF']) 
                )

            entry_buy_score /= (2*self.WEIGHT_DAILY + 2*self.WEIGHT_4HOUR + self.WEIGHT_15MIN + 1)*(3)

            entry_sell_score = (
                    (trend_score_daily < -0.5) * self.WEIGHT_DAILY + \
                    ((trend_score_daily > 0.5) & (pivot_score_daily > 0.5)) * self.WEIGHT_DAILY + \
                    (trend_score_4hour < -0.3) * self.WEIGHT_4HOUR + \
                    ((trend_score_4hour > 0.3) & (pivot_score_4hour > 0.3)) * self.WEIGHT_4HOUR + \
                    (trend_score_15min < 0) * self.WEIGHT_15MIN + \
                    1
                ) * \
                (
                    (row['RSI'] > self.RSI_PARAMS['CUTOFF_SELL']) + \
                    (
                        (row['CLOSE'] >= row[f'EMA({self.EMA_1_PARAMS["INPUT_PARAM"]})']) & \
                        (row['CLOSE'] <= row[f'EMA({self.EMA_2_PARAMS["INPUT_PARAM"]})'])
                    ) + \
                    (np.abs(row['CLOSE'] - row[f'BB_UPPER_BAND({self.BB_PARAMS["INPUT_MEAN"]})']) < self.BB_PARAMS['BB_CUTOFF']) 
                )

            entry_sell_score /= (2*self.WEIGHT_DAILY + 2*self.WEIGHT_4HOUR + self.WEIGHT_15MIN + 1)*(3)
            
            signal = 0
            if ((entry_buy_score > self.DECISION_CUTOFF['ENTRY_BUY_CUTOFF']) & (entry_buy_score > entry_sell_score)):
                signal = 1
                SL = self.base_SL
                TP = self.base_TP
            elif ((entry_sell_score > self.DECISION_CUTOFF['ENTRY_SELL_CUTOFF']) & (entry_sell_score > entry_buy_score)):
                signal = -1
                SL = self.base_SL
                TP = self.base_TP
            else: 
                signal = 0
                SL = 0
                TP = 0
            

            return(pd.Series([trend_score_daily, 
                            pivot_score_daily, 
                            trend_score_4hour, 
                            pivot_score_4hour, 
                            trend_score_15min, 
                            pivot_score_15min, 
                            entry_buy_score, 
                            entry_sell_score,
                            signal,
                            SL,
                            TP
                            ], 
                            index = [
                                'TREND_SCORE_DAILY',
                                'PIVOT_SCORE_DAILY',
                                'TREND_SCORE_4HOUR',
                                'PIVOT_SCORE_4HOUR',
                                'TREND_SCORE_15MIN',
                                'PIVOT_SCORE_15MIN',
                                'ENTRY_BUY_SCORE',
                                'ENTRY_SELL_SCORE',
                                'SIGNAL', 
                                'SL',
                                'TP'
                            ]))
        except:
            return(pd.Series([None, 
                            None, 
                            None, 
                            None, 
                            None, 
                            None, 
                            None, 
                            None,
                            0,
                            0,
                            0
                            ], 
                            index = [
                                'TREND_SCORE_DAILY',
                                'PIVOT_SCORE_DAILY',
                                'TREND_SCORE_4HOUR',
                                'PIVOT_SCORE_4HOUR',
                                'TREND_SCORE_15MIN',
                                'PIVOT_SCORE_15MIN',
                                'ENTRY_BUY_SCORE',
                                'ENTRY_SELL_SCORE',
                                'SIGNAL', 
                                'SL', 
                                'TP'
                            ]))



## In sample optimization

In [ ]:
config = {
    'RSI_PARAMS': {'INPUT_PARAM': 7, 'CUTOFF_BUY': 30, 'CUTOFF_SELL': 70},
    'BB_PARAMS': {'INPUT_MEAN': 50, 'INPUT_SD': 2, 'BB_CUTOFF': 3},
    'EMA_1_PARAMS': {'INPUT_PARAM': 50},
    'EMA_2_PARAMS': {'INPUT_PARAM': 200},
    'WEIGHT_DAILY': 3,
    'WEIGHT_4HOUR': 2,
    'WEIGHT_15MIN': 2,
    'LOOKBACK': 20,
    'base_SL': 10, 
    'base_TP': 20,
    'DECISION_CUTOFF': {'ENTRY_BUY_CUTOFF': 0.2, 'ENTRY_SELL_CUTOFF': 0.2}
}

alpha = TA_strat_v1(config = config)

In [ ]:
df_15_min_is.shape

In [ ]:
cnt = 0

while cnt <= 10:
    print('='*100)
    print(cnt)
    print('='*100)
    id = random.randint(0, df_15_min_is.shape[0])
    report_is = Backtest_report(alpha = alpha,
                                df_is = df_15_min_is[id - 50: id + 5000].copy(),
                                base_SL = 10,
                                base_TP = 20,
                                max_existing_positions = 5,
                                init_vol = 0.01,
                                re_allocation = True,
                                incre_vol = 0.01,
                                max_vol = 0.1,
                                init_cap = 1000,
                                incre_cap = 2
                                )

    report_is.display_report()
    cnt += 1

In [ ]:
# report_is.prepare_report()

In [ ]:
df_result_is = report_is.df_result_is
df_balance_is = report_is.df_balance_is
df_balance_new = report_is.df_balance_new


In [ ]:
df_result_is['CLOSE'] = df_result_is['CLOSE_open'].copy()

In [ ]:
df_summary = pd.concat(
    [
        pd.pivot_table(
            report_is.df_result_is[(report_is.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_is.df_result_is['SIGNAL'] != 0) & (report_is.df_result_is['PNL'] > 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        ),
        pd.pivot_table(
            report_is.df_result_is[(report_is.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_is.df_result_is['SIGNAL'] != 0) & (report_is.df_result_is['PNL'] < 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        )
    ],
    axis = 1
)

df_summary.columns = ['WINNING_POSITIONS', 'LOSING_POSITIONS']
df_summary.index = ['SHORT', 'LONG', 'ALL']
df_summary = df_summary.fillna(0)
df_summary['TOTAL_POSITIONS'] = df_summary['WINNING_POSITIONS'] + df_summary['LOSING_POSITIONS']

In [ ]:
df_summary

In [ ]:
df_result_is[['TREND_SCORE_DAILY', 'PIVOT_SCORE_DAILY', 
              'TREND_SCORE_4HOUR', 'PIVOT_SCORE_4HOUR', 
              'TREND_SCORE_15MIN', 'PIVOT_SCORE_15MIN']].hist()

In [ ]:
df_result_is[['ENTRY_BUY_SCORE', 'ENTRY_SELL_SCORE']].hist()

### Tuning

In [ ]:
new_config = {
    'RSI_PARAMS': {'INPUT_PARAM': 7, 'CUTOFF_BUY': 30, 'CUTOFF_SELL': 70},
    'BB_PARAMS': {'INPUT_MEAN': 50, 'INPUT_SD': 2, 'BB_CUTOFF': 3},
    'EMA_1_PARAMS': {'INPUT_PARAM': 50},
    'EMA_2_PARAMS': {'INPUT_PARAM': 200},
    'WEIGHT_DAILY': 3,
    'WEIGHT_4HOUR': 2,
    'WEIGHT_15MIN': 2,
    'LOOKBACK': 20,
    'base_SL': 10, 
    'base_TP': 20,
    'DECISION_CUTOFF': {'ENTRY_BUY_CUTOFF': 0.02, 'ENTRY_SELL_CUTOFF': 0.02}
}

In [ ]:
def _TA_strat_v1_test(row):
    RSI_PARAMS = new_config['RSI_PARAMS']
    BB_PARAMS = new_config['BB_PARAMS']
    EMA_1_PARAMS = new_config['EMA_1_PARAMS']
    EMA_2_PARAMS = new_config['EMA_2_PARAMS']
    WEIGHT_DAILY = new_config['WEIGHT_DAILY']
    WEIGHT_4HOUR = new_config['WEIGHT_4HOUR']
    WEIGHT_15MIN = new_config['WEIGHT_15MIN']
    base_SL = new_config['base_SL']
    base_TP = new_config['base_TP']
    DECISION_CUTOFF = new_config['DECISION_CUTOFF']
    
    entry_buy_score = (
            (row['TREND_SCORE_DAILY'] > 0.3) * WEIGHT_DAILY + \
            ((row['TREND_SCORE_DAILY'] < -0.3) & (row['PIVOT_SCORE_DAILY'] > 0.3)) * WEIGHT_DAILY + \
            (row['TREND_SCORE_4HOUR'] > 0.2) * WEIGHT_4HOUR + \
            ((row['TREND_SCORE_4HOUR'] < -0.2) & (row['PIVOT_SCORE_4HOUR'] > 0.2)) * WEIGHT_4HOUR + \
            (row['TREND_SCORE_15MIN'] > 0) * WEIGHT_15MIN + \
            1
        ) * \
        (
            (row['RSI'] < RSI_PARAMS['CUTOFF_BUY']) + \
            (
                (row['CLOSE'] <= row[f'EMA({EMA_1_PARAMS["INPUT_PARAM"]})']) & \
                (row['CLOSE'] >= row[f'EMA({EMA_2_PARAMS["INPUT_PARAM"]})'])
            ) + \
            (np.abs(row['CLOSE'] - row[f'BB_LOWER_BAND({BB_PARAMS["INPUT_MEAN"]})']) < BB_PARAMS['BB_CUTOFF']) 
        )

    entry_buy_score /= (2*WEIGHT_DAILY + 2*WEIGHT_4HOUR + WEIGHT_15MIN + 1)*(3)

    entry_sell_score = (
            (row['TREND_SCORE_DAILY'] < -0.3) * WEIGHT_DAILY + \
            ((row['TREND_SCORE_DAILY'] > 0.3) & (row['PIVOT_SCORE_DAILY'] > 0.3)) * WEIGHT_DAILY + \
            (row['TREND_SCORE_4HOUR'] < -0.2) * WEIGHT_4HOUR + \
            ((row['TREND_SCORE_4HOUR'] > 0.2) & (row['PIVOT_SCORE_4HOUR'] > 0.2)) * WEIGHT_4HOUR + \
            (row['TREND_SCORE_15MIN'] < 0) * WEIGHT_15MIN + \
            1
        ) * \
        (
            (row['RSI'] > RSI_PARAMS['CUTOFF_SELL']) + \
            (
                (row['CLOSE'] >= row[f'EMA({EMA_1_PARAMS["INPUT_PARAM"]})']) & \
                (row['CLOSE'] <= row[f'EMA({EMA_2_PARAMS["INPUT_PARAM"]})'])
            ) + \
            (np.abs(row['CLOSE'] - row[f'BB_UPPER_BAND({BB_PARAMS["INPUT_MEAN"]})']) < BB_PARAMS['BB_CUTOFF']) 
        )

    entry_sell_score /= (2*WEIGHT_DAILY + 2*WEIGHT_4HOUR + WEIGHT_15MIN + 1)*(3)
    
    signal = 0
    if ((entry_buy_score > DECISION_CUTOFF['ENTRY_BUY_CUTOFF']) & (entry_buy_score > entry_sell_score)):
        signal = 1
        SL = base_SL
        TP = base_TP
    elif ((entry_sell_score > DECISION_CUTOFF['ENTRY_SELL_CUTOFF']) & (entry_sell_score > entry_buy_score)):
        signal = -1
        SL = base_SL
        TP = base_TP
    else: 
        signal = 0
        SL = 0
        TP = 0
    

    return(pd.Series([
                    entry_buy_score, 
                    entry_sell_score,
                    signal,
                    SL,
                    TP
                    ], 
                    index = [
                        'ENTRY_BUY_SCORE',
                        'ENTRY_SELL_SCORE',
                        'SIGNAL', 
                        'SL',
                        'TP'
                    ]))


In [ ]:
new_signal = df_result_is.apply(_TA_strat_v1_test, axis = 1)

In [ ]:
new_signal[['ENTRY_BUY_SCORE', 'ENTRY_SELL_SCORE']].hist()

## Out-sample test

In [ ]:
cnt = 0

while cnt <= 10:
    print('='*100)
    print(cnt)
    print('='*100)
    id = random.randint(0, df_15_min_os.shape[0])
    report_os = Backtest_report(alpha = alpha,
                                df_is = df_15_min_os[id - 50: id + 5000].copy(),
                                base_SL = 10,
                                base_TP = 20,
                                max_existing_positions = 5,
                                init_vol = 0.01,
                                re_allocation = True,
                                incre_vol = 0.01,
                                max_vol = 0.1,
                                init_cap = 1000,
                                incre_cap = 2
                                )

    report_os.display_report()
    cnt += 1

In [ ]:
pd.concat(
    [
        pd.pivot_table(
            report_os.df_result_is[(report_os.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_os.df_result_is['SIGNAL'] != 0) & (report_os.df_result_is['PNL'] > 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        ),
        pd.pivot_table(
            report_os.df_result_is[(report_os.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_os.df_result_is['SIGNAL'] != 0) & (report_os.df_result_is['PNL'] < 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        )
    ],
    axis = 1
)

In [ ]:
df_summary = pd.concat(
    [
        pd.pivot_table(
            report_os.df_result_is[(report_os.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_os.df_result_is['SIGNAL'] != 0) & (report_os.df_result_is['PNL'] > 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        ),
        pd.pivot_table(
            report_os.df_result_is[(report_os.df_result_is['FLAG_VALID_POSITION'] == 1) & (report_os.df_result_is['SIGNAL'] != 0) & (report_os.df_result_is['PNL'] < 0)],
            index = 'SIGNAL',
            values = 'CLOSE_open',
            aggfunc = 'count',
            margins = True
        )
    ],
    axis = 1
)

df_summary.columns = ['WINNING_POSITIONS', 'LOSING_POSITIONS']
df_summary.index = ['SHORT', 'LONG', 'ALL']
df_summary = df_summary.fillna(0)
df_summary['TOTAL_POSITIONS'] = df_summary['WINNING_POSITIONS'] + df_summary['LOSING_POSITIONS']

In [ ]:
df_summary